## Advent of code 2019 day 11-20
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 13
# start_ts=1665212872
# mv ~/Downloads/input* data_src/2019-day-13-input.txt
# big input file looks like: intcode with serious data included
# idea: part 1 parse/run intcode, draw outputs, then count

#sample1='''
#'''

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data):
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self):
        assert self.board is None
        self.board={} # maps (x,y) cell to tile id
        self.out_q=[]
        self.turns=0
        self.score=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        #if self.turns%100==0:
        #    self.print_painted()
        #if self.turns>1000:
        #    exit()
        self.turns+=1
        return self.best_move()

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        self.out_q.append(a)
        if len(self.out_q)>=3:
            x,y,tid=self.out_q
            if x== -1 and y==0:
                self.score=tid
            else:
                self.board[ (x, y) ]=tid
            self.out_q=[]

    def count_painted(self):
        return len([ tid for tid in self.board.values() if tid == 2])

    def print_painted(self):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , 0)
                row+=str(c) if c!=0 else ' '
            print(row)

    def best_move(self):
        '''move towards the ball'''
        ball_x=None
        paddle_x=None
        for tup,tid in self.board.items():
            if tid==4:
                ball_x=tup[0]
            elif tid==3:
                paddle_x=tup[0]
        assert ball_x is not None
        assert paddle_x is not None
        if ball_x>paddle_x:
            return 1
        elif ball_x<paddle_x:
            return -1
        else:
            return 0

sample1=open('data_src/2019-day-13-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board()
    if do_part==2:
        data[0]=2
    cmp.run_opcodes(data)
    if do_part==1:
        print(cmp.count_painted())
    else:
        cmp.print_painted()
        print(f'won? {cmp.score=}')

# part 1: 200
# part 2: 9803

In [ ]:
# 2019 day 12 part 1
# start_ts=1664775498
# mv ~/Downloads/input* data_src/2019-day-12-input.txt
# big input file looks like: 4 xyz coords -8 .. 17
# idea: part 1 parse w/re, then simulate

sample1='''
<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>
'''

sample2='''
<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>
'''

def sign(n):
    if n<0:
        return -1
    elif n>0:
        return 1
    else:
        return 0

def do_gravity(pos_data, vel_data):
    for i in range(len(pos_data)):
        for j in range(i+1, len(pos_data)):
            for k in range(3):
                dist=pos_data[i][k]-pos_data[j][k]
                vel_data[i][k]-=sign(dist)
                vel_data[j][k]+=sign(dist)

def do_velocity(pos_data, vel_data):
    for i in range(len(pos_data)):
        for k in range(3):
            pos_data[i][k]+=vel_data[i][k]

def energy(pos_data, vel_data):
    res=0
    for i in range(len(pos_data)):
        pot=sum([abs(n) for n in pos_data[i]])
        kin=sum([abs(n) for n in vel_data[i]])
        res+=pot*kin
    return res

def unlist(tl):
    res=[]
    last=tl[0]
    for n in tl[1:]:
        res.append(n-last)
        last=n
    return tl[0], res

def do_universe(pos_data, vel_data, universe, turn):
    for k in range(3):
        posl=[ll[k] for ll in pos_data]
        vell=[ll[k] for ll in vel_data]
        posvel=tuple(posl+vell)
        tl=universe[k].setdefault( posvel, [])
        tl.append(turn)

sample1=open('data_src/2019-day-12-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
pos_data=[ [int(gs) for gs in result.group(1, 2, 3)] for s in lines if (result:= re.match(r'.*x=([\d-]+)\s*,\s*y=([\d-]+)\s*,\s*z=([\d-]+)', s)) ]
vel_data=[]
for i in pos_data:
    vel_data.append( [0, 0, 0] )
universe=[{}, {}, {}] # list of maps of (pos-m0 pos-m1.. vel-m0 vel-m1) to [turns], i.e. per coord
do_universe(pos_data, vel_data, universe, 0)
for stepno in range(1000):
    do_gravity(pos_data, vel_data)
    do_velocity(pos_data, vel_data)
    do_universe(pos_data, vel_data, universe, stepno+1)
print('energy:', energy(pos_data, vel_data))

# part 1: 9876

In [ ]:
# part 2
# idea: part 2 record list of states separately per x, y and z coords (so e.g. x0 x1 x2 x3 vx0 vx1 vx2 vx3)
# with the turns in which they come up, then you can see their periodicity, apparently every cycle has the same
# length looking at one of the x/y/z components, so the earliest position is the starting position (proof left
# for the reader), then just calculate the least common multiple

# continue running until universe is filled enough so that every coord has repeated at least once
while stepno<1000000:
    stepno+=1
    do_gravity(pos_data, vel_data)
    do_velocity(pos_data, vel_data)
    do_universe(pos_data, vel_data, universe, stepno+1)
periods={} # maps k to first period
for k in range(3):
    print(f'coord {k}')
    kpers=set()
    for st, perl in [unlist(tl) for tl in universe[k].values()]:
        kpers.update(set(perl))
    print(f'{kpers=}')
    assert len(kpers)==1 # apparently always(?) only one period per coordinate
    periods[k]=min(kpers)
vals=list(periods.values())
assert len(vals)==3
lcm1=vals[0]*vals[1]//math.gcd(vals[0], vals[1])
lcm2=lcm1*vals[2]//math.gcd(lcm1, vals[2])
print(f'{lcm2=}')

# part 2: 307043147758488

In [ ]:
# 2019 day 11
# start_ts=1664690374
# mv ~/Downloads/input* data_src/2019-day-11-input.txt
# big input file looks like: intcode
# idea: part 1 parse & run intcode, then manage input/output as a board of coords starting at 0,0 with color None

#sample1='''
#'''

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data):
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self):
        assert self.board is None
        self.board={} # maps (x,y) cell to color 0 or 1
        self.rob_x=0
        self.rob_y=0
        self.rob_dir=0 # 0 is up, 1 is right, 2 is down, 3 is left
        self.rob_painting=True # if True output will paint, if False output will turn and move

    def get_input(self):
        if self.board is None:
            self.init_board()
        return self.board.get( (self.rob_x, self.rob_y) , 0)

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        assert 0 <= a <= 1
        if self.rob_painting:
            self.board[ (self.rob_x, self.rob_y) ]=a
            self.rob_painting=False
        else: # turn & moving
            if a==0:
                self.rob_dir-=1
            else:
                self.rob_dir+=1
            self.rob_dir=self.rob_dir % 4
            if self.rob_dir==0:
                self.rob_y-=1
            elif self.rob_dir==1:
                self.rob_x+=1
            elif self.rob_dir==2:
                self.rob_y+=1
            elif self.rob_dir==3:
                self.rob_x-=1
            else:
                assert False
            self.rob_painting=True

    def count_painted(self):
        return len(self.board)

    def print_painted(self):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , 0)
                row+='X' if c==1 else ' '
            print(row)

sample1=open('data_src/2019-day-11-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1, 2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board()
    if do_part==2:
        cmp.board[ (0,0) ]=1
    cmp.run_opcodes(data)
    if do_part==1:
        print(cmp.count_painted())
    elif do_part==2:
        cmp.print_painted()

# part 1: 2293
# part 2: AHLCPRAL


In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample1='''

'''

#sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed